## Attribute generation for the PRC Challenge 2025


In [1]:
import pandas as pd
import numpy as np
import pickle
import time
from tqdm import tqdm

In [2]:
!pip3 install pyarrow fastparquet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 909.9/909.9 kB 756.6 kB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 997.3 kB/s eta 0:00:00:0100:010m


In [3]:
folder = '~/OneDrive - Embry-Riddle Aeronautical University/PRC Challenge 2025 - challenge data/final_phase/prc-2025-datasets/'
df_fuel = pd.read_parquet(folder+'fuel_train.parquet')
df_airpt = pd.read_parquet(folder+'apt.parquet')
df_flight = pd.concat([
    pd.read_parquet(folder+'flightlist_train.parquet').assign(dataset='train'), 
    pd.read_parquet(folder+'flightlist_rank.parquet').assign(dataset='rank'),
    pd.read_parquet(folder+'flightlist_final.parquet').assign(dataset='final')],
    ignore_index=True
)
df_fuel_rank = pd.read_parquet(folder+'fuel_final_submission.parquet')

In [4]:
df_flight

,flight_date,aircraft_type,takeoff,landed,origin_icao,origin_name,destination_icao,destination_name,flight_id,dataset
0,2025-04-12,B789,2025-04-12 16:15:35,2025-04-13 04:33:52,WMKK,Kuala Lumpur International Airport,EHAM,Amsterdam Schiphol Airport,prc770822360,train
1,2025-04-12,A359,2025-04-12 16:40:41,2025-04-13 03:10:15,ZSPD,Shanghai Pudong International Airport,NZAA,Auckland Airport,prc770831136,train
2,2025-04-12,A359,2025-04-12 17:01:38,2025-04-13 06:10:27,ZSPD,Shanghai Pudong International Airport,LEMD,Madrid Barajas Airport,prc770835414,train
3,2025-04-12,B789,2025-04-12 17:53:11,2025-04-13 05:05:13,ZSPD,Shanghai Pudong International Airport,LHBP,Budapest Ferenc Liszt International Airport,prc770844923,train
4,2025-04-12,B789,2025-04-12 18:05:27,2025-04-13 06:23:52,ZSPD,Shanghai Pudong International Airport,LFML,Marseille Provence Airport,prc770847190,train
...,...,...,...,...,...,...,...,...,...,...
15756,2025-10-31,A20N,2025-10-31 20:57:23,2025-11-01 00:31:01,KMDW,Chicago Midway International Airport,MMLO,Silao Del Bajio International Airport,prc821901646,final
15757,2025-10-31,A20N,2025-10-31 21:26:00,2025-11-01 03:39:56,SAEZ,Buenos Aires Ministro Pistarini International ...,SKBO,Bogota El Dorado International Airport,prc821906723,final
15758,2025-10-31,A320,2025-10-31 21:45:04,2025-11-01 02:01:56,KIAD,Washington Dulles International Airport,MSLP,San Salvador International Airport,prc821909226,final
15759,2025-10-31,A320,2025-10-31 22:06:32,2025-11-01 03:02:30,KPHL,Philadelphia International Airport,KLAS,Las Vegas Harry Reid International Airport,prc821914241,final


In [5]:
#inspection of flightlist files
df_flight.head()


,flight_date,aircraft_type,takeoff,landed,origin_icao,origin_name,destination_icao,destination_name,flight_id,dataset
0,2025-04-12,B789,2025-04-12 16:15:35,2025-04-13 04:33:52,WMKK,Kuala Lumpur International Airport,EHAM,Amsterdam Schiphol Airport,prc770822360,train
1,2025-04-12,A359,2025-04-12 16:40:41,2025-04-13 03:10:15,ZSPD,Shanghai Pudong International Airport,NZAA,Auckland Airport,prc770831136,train
2,2025-04-12,A359,2025-04-12 17:01:38,2025-04-13 06:10:27,ZSPD,Shanghai Pudong International Airport,LEMD,Madrid Barajas Airport,prc770835414,train
3,2025-04-12,B789,2025-04-12 17:53:11,2025-04-13 05:05:13,ZSPD,Shanghai Pudong International Airport,LHBP,Budapest Ferenc Liszt International Airport,prc770844923,train
4,2025-04-12,B789,2025-04-12 18:05:27,2025-04-13 06:23:52,ZSPD,Shanghai Pudong International Airport,LFML,Marseille Provence Airport,prc770847190,train


In [6]:
df_airpt.head()

,icao,longitude,latitude,elevation
0,HCAD,46.637500,6.095802,1001.0
1,OAAK,65.206746,36.942659,900.0
2,OALG,69.023102,33.986500,6333.0
3,OAMK,67.848602,32.879601,6600.0
4,OAPJ,67.020445,34.391726,8800.0


In [7]:
df_fuel.head()

,idx,flight_id,start,end,fuel_kg
0,0,prc770822360,2025-04-13 02:31:04.447,2025-04-13 03:01:04.487,2500.0
1,1,prc770822360,2025-04-13 03:01:04.487,2025-04-13 03:16:04.226,1200.0
2,2,prc770822360,2025-04-13 03:16:04.458,2025-04-13 03:46:04.417,2500.0
3,3,prc770822360,2025-04-13 03:46:04.417,2025-04-13 04:01:04.540,1300.0
4,4,prc770822360,2025-04-13 04:01:04.540,2025-04-13 04:16:04.263,600.0


In [8]:
df_fuel_rank.head()

,idx,flight_id,start,end,fuel_kg
0,0,prc806615763,2025-09-01 03:03:10.925,2025-09-01 03:07:51.584,None
1,1,prc806615763,2025-09-01 03:07:51.584,2025-09-01 03:12:50.921,None
2,2,prc806615763,2025-09-01 03:12:50.921,2025-09-01 03:17:51.404,None
3,3,prc806615763,2025-09-01 03:17:51.404,2025-09-01 03:22:50.539,None
4,4,prc806615763,2025-09-01 03:22:50.539,2025-09-01 03:27:50.727,None


In [9]:
df_fuel = pd.concat(
    [df_fuel.assign(seg_dataset='train'), df_fuel_rank.assign(seg_dataset='final')],
    ignore_index=True
)

/var/folders/j8/hxyhjpsn0mz62n28qrbhdtyh0000gn/T/ipykernel_2150/2372689071.py:1: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_fuel = pd.concat(


In [10]:
df_fuel

,idx,flight_id,start,end,fuel_kg,seg_dataset
0,0,prc770822360,2025-04-13 02:31:04.447,2025-04-13 03:01:04.487,2500.0,train
1,1,prc770822360,2025-04-13 03:01:04.487,2025-04-13 03:16:04.226,1200.0,train
2,2,prc770822360,2025-04-13 03:16:04.458,2025-04-13 03:46:04.417,2500.0,train
3,3,prc770822360,2025-04-13 03:46:04.417,2025-04-13 04:01:04.540,1300.0,train
4,4,prc770822360,2025-04-13 04:01:04.540,2025-04-13 04:16:04.263,600.0,train
...,...,...,...,...,...,...
193270,37451,prc821916593,2025-10-31 23:06:58.703,2025-10-31 23:17:08.526,NaN,final
193271,37452,prc821916593,2025-10-31 23:17:08.526,2025-10-31 23:22:13.468,NaN,final
193272,37453,prc821916593,2025-10-31 23:22:13.468,2025-10-31 23:27:18.458,NaN,final
193273,37454,prc821916593,2025-10-31 23:27:18.458,2025-10-31 23:52:43.483,NaN,final


In [11]:
#inspection of trajectory files

# flight_id = 'prc770822360'
flight_id = 'prc770831136'
df_single_flight = pd.read_parquet(folder + 'flights_train/' + flight_id + '.parquet')

df_single_flight.head()

,timestamp,flight_id,typecode,latitude,longitude,altitude,groundspeed,track,vertical_rate,mach,TAS,CAS,source
0,2025-04-12 21:30:33.782,prc770831136,A359,4.03,151.76,37006.0,NaN,NaN,NaN,NaN,NaN,277.0,acars
1,2025-04-12 21:58:39.507,prc770831136,A359,0.84,153.69,37006.0,NaN,NaN,NaN,NaN,NaN,279.0,acars
2,2025-04-12 22:02:40.552,prc770831136,A359,0.38,153.97,37006.0,NaN,NaN,NaN,NaN,NaN,278.0,acars
3,2025-04-12 22:22:44.881,prc770831136,A359,-1.87,155.33,37002.0,NaN,NaN,NaN,NaN,NaN,278.0,acars
4,2025-04-12 22:38:54.572,prc770831136,A359,-3.70,156.37,37001.0,NaN,NaN,NaN,NaN,NaN,273.0,acars


# Augment flight dataset

In [12]:
df_flight = df_flight.merge(df_airpt.add_prefix('origin_'), left_on='origin_icao', right_on='origin_icao', how='left')
df_flight = df_flight.merge(df_airpt.add_prefix('destination_'), left_on='destination_icao', right_on='destination_icao', how='left')

In [13]:
from numpy import radians, sin, cos, sqrt, arctan2

def haversine(lat1, lon1, lat2, lon2):
    # Calculate great circle distance between two points on Earth (in km)
    R = 6371.0  # Earth radius in kilometers
    lat1, lon1, lat2, lon2 = map(radians, [lat1, lon1, lat2, lon2])
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * arctan2(sqrt(a), sqrt(1-a))
    return R * c

df_flight['great_circle_distance_km'] = haversine(
    df_flight['origin_latitude'],
    df_flight['origin_longitude'],
    df_flight['destination_latitude'],
    df_flight['destination_longitude']
)

In [14]:
def haversine_vectorized(lat1, lon1, lat2, lon2, radius_km=6371.0):
    """
    Vectorized haversine (returns km). Inputs are arrays/Series in degrees.
    Any input NaNs will result in NaN output for that element.
    """
    # convert to radians
    lat1r = np.radians(lat1)
    lon1r = np.radians(lon1)
    lat2r = np.radians(lat2)
    lon2r = np.radians(lon2)

    dlat = lat2r - lat1r
    dlon = lon2r - lon1r

    a = np.sin(dlat / 2.0)**2 + np.cos(lat1r) * np.cos(lat2r) * np.sin(dlon / 2.0)**2
    # numeric safety for small negatives due to floating error
    a = np.clip(a, 0.0, 1.0)
    c = 2.0 * np.arcsin(np.sqrt(a))

    # where any input is NaN, produce NaN (propagate)
    invalid = pd.isna(lat1) | pd.isna(lon1) | pd.isna(lat2) | pd.isna(lon2)
    dist = radius_km * c
    dist = dist.astype(float)
    dist[invalid] = np.nan
    return dist

def add_haversine_to_prev(df, flight_id_col='flight_id', time_col='timestamp',
                          lat_col='latitude', lon_col='longitude',
                          out_col='haversine_to_prev_km', inplace=False):
    """
    Sorts by [flight_id_col, time_col], computes haversine distance from previous
    row *within the same flight_id*, and writes a column with distances (km).
    The first row of each flight_id will be NaN (no previous point).
    Returns the modified dataframe (a copy unless inplace=True).
    """
    if not inplace:
        df = df.copy()

    # ensure timestamp is datetime for correct sorting (no-op if already datetime)
    if not np.issubdtype(df[time_col].dtype, np.datetime64):
        df[time_col] = pd.to_datetime(df[time_col])

    # sort by flight_id then timestamp
    df = df.sort_values([flight_id_col, time_col], kind='mergesort')  # stable sort

    # shift lat/lon by 1 within each flight_id
    prev_lat = df.groupby(flight_id_col)[lat_col].shift(1)
    prev_lon = df.groupby(flight_id_col)[lon_col].shift(1)

    # current lat/lon
    cur_lat = df[lat_col]
    cur_lon = df[lon_col]

    # compute vectorized haversine
    df[out_col] = haversine_vectorized(prev_lat, prev_lon, cur_lat, cur_lon)
    
    # fill NaN with 0.0 (first point of each flight_id)
    df[out_col] = df[out_col].fillna(0.0)

    # Optionally, keep original order index if you want to restore original indexing:
    # df = df.sort_index()

    return df

# Usage:
# df_new = add_haversine_to_prev(df_flight_train)
# print(df_new[['light_id','timestamp','latitude','longitude','haversine_to_prev_km']].head(10))


The following step generates features related to the trajectory file. It took about 20 minutes to run.

In [15]:
# ensure columns exists in flight df
if 'total_flight_distance_km' not in df_flight.columns:
    df_flight['total_flight_distance_km'] = 0.0
if 'altitude_max' not in df_flight.columns:
    df_flight['altitude_max'] = 0.0
if 'altitude_min' not in df_flight.columns:
    df_flight['altitude_min'] = 0.0
if 'altitude_diff' not in df_flight.columns:
    df_flight['altitude_diff'] = 0.0
if 'avg_rate_of_climb' not in df_flight.columns:
    df_flight['avg_rate_of_climb'] = 0.0
if 'sd_rate_of_climb' not in df_flight.columns:
    df_flight['sd_rate_of_climb'] = 0.0

# ensure columns exists in fuel df
if 'seg_flight_distance_km' not in df_fuel.columns:
    df_fuel['seg_flight_distance_km'] = 0.0
if 'seg_altitude_max' not in df_fuel.columns:
    df_fuel['seg_altitude_max'] = 0.0
if 'seg_altitude_min' not in df_fuel.columns:
    df_fuel['seg_altitude_min'] = 0.0
if 'seg_altitude_diff' not in df_fuel.columns:
    df_fuel['seg_altitude_diff'] = 0.0
if 'seg_avg_rate_of_climb' not in df_fuel.columns:
    df_fuel['seg_avg_rate_of_climb'] = 0.0
if 'seg_sd_rate_of_climb' not in df_fuel.columns:
    df_fuel['seg_sd_rate_of_climb'] = 0.0
if 'seg_pos_count' not in df_fuel.columns:
    df_fuel['seg_pos_count'] = 0.0


# loop over train flights to calculate additional features
akk = 0

for item in df_flight.itertuples():
    akk += 1
    flight_id = item.flight_id
    print(flight_id, akk," of ",len(df_flight))

    df_flight_train = add_haversine_to_prev(
        pd.read_parquet(folder + 'flights_'+item.dataset+f'/{flight_id}.parquet')
    )


    total = df_flight_train['haversine_to_prev_km'].sum() # total distance for flight
    max_alt = df_flight_train['altitude'].max() # max altitude for flight
    min_alt = df_flight_train['altitude'].min() # min altitude for flight
    alt_diff = max_alt - min_alt # altitude difference for flight
    df_flight.loc[df_flight['flight_id'] == flight_id, 'total_flight_distance_km'] = total # total distance for flight
    df_flight.loc[df_flight['flight_id'] == flight_id, 'altitude_max'] = max_alt # max altitude for flight
    df_flight.loc[df_flight['flight_id'] == flight_id, 'altitude_min'] = min_alt # min altitude for flight
    df_flight.loc[df_flight['flight_id'] == flight_id, 'altitude_diff'] = alt_diff # altitude difference for flight
    df_flight.loc[df_flight['flight_id'] == flight_id, 'avg_rate_of_climb'] = df_flight_train['vertical_rate'].mean() # average rate of climb for flight
    df_flight.loc[df_flight['flight_id'] == flight_id, 'sd_rate_of_climb'] = df_flight_train['vertical_rate'].std() # standard deviation of rate of climb for flight

    # loop over flight segments to calculate additional features
    for seg in df_fuel[df_fuel['flight_id'] == flight_id].itertuples():
        # print("   segment ", seg.idx, seg.start, " to ", seg.end)
        df_flight_train_seg = df_flight_train[
            (df_flight_train['timestamp'] >= seg.start) &
            (df_flight_train['timestamp'] <= seg.end)
        ]
        df_fuel.loc[(df_fuel['idx'] == seg.idx), 'seg_flight_distance_km'] = df_flight_train_seg['haversine_to_prev_km'].iloc[1:].sum() # calculate distance flown per segment excluding first point
        seg_max_alt = df_flight_train_seg['altitude'].max() # max altitude for segment
        seg_min_alt = df_flight_train_seg['altitude'].min() # min altitude for segment
        seg_alt_diff = seg_max_alt - seg_min_alt # altitude difference for segment
        df_fuel.loc[(df_fuel['idx'] == seg.idx), 'seg_altitude_max'] = seg_max_alt # max altitude for segment
        df_fuel.loc[(df_fuel['idx'] == seg.idx), 'seg_altitude_min'] = seg_min_alt # min altitude for segment
        df_fuel.loc[(df_fuel['idx'] == seg.idx), 'seg_altitude_diff'] = seg_alt_diff # altitude difference for segment
        df_fuel.loc[(df_fuel['idx'] == seg.idx), 'seg_avg_rate_of_climb'] = df_flight_train_seg['vertical_rate'].mean() # average rate of climb for segment
        df_fuel.loc[(df_fuel['idx'] == seg.idx), 'seg_sd_rate_of_climb'] = df_flight_train_seg['vertical_rate'].std() # standard deviation of rate of climb for segment
        df_fuel.loc[(df_fuel['idx'] == seg.idx), 'seg_pos_count'] = df_flight_train_seg['timestamp'].count() # count of available positions per segment

        # print("       ",  df_flight_train_seg['haversine_to_prev_km'].iloc[1:].sum(), df_flight_train_seg['timestamp'].count())





prc770822360 1  of  15761
prc770831136 2  of  15761
prc770835414 3  of  15761
prc770844923 4  of  15761
prc770847190 5  of  15761
prc770852254 6  of  15761
prc770853645 7  of  15761
prc770859206 8  of  15761
prc770860078 9  of  15761
prc770864956 10  of  15761
prc770866486 11  of  15761
prc770867379 12  of  15761
prc770868424 13  of  15761
prc770870642 14  of  15761
prc770872129 15  of  15761
prc770875579 16  of  15761
prc770876146 17  of  15761
prc770876159 18  of  15761
prc770876744 19  of  15761
prc770878608 20  of  15761
prc770879349 21  of  15761
prc770881363 22  of  15761
prc770882585 23  of  15761
prc770882767 24  of  15761
prc770885136 25  of  15761
prc770887177 26  of  15761
prc770887278 27  of  15761
prc770887555 28  of  15761
prc770887933 29  of  15761
prc770888915 30  of  15761
prc770893597 31  of  15761
prc770894018 32  of  15761
prc770896208 33  of  15761
prc770897620 34  of  15761
prc770900396 35  of  15761
prc770900987 36  of  15761
prc770901870 37  of  15761
prc7709032

In [16]:
df_flight_train

,timestamp,flight_id,typecode,latitude,longitude,altitude,groundspeed,track,vertical_rate,mach,TAS,CAS,source,haversine_to_prev_km
0,2025-10-31 22:23:02.762000084,prc821916593,A20N,41.800415,-87.772584,1650.0,169.0,314.760270,1536.0,NaN,NaN,NaN,adsb,0.000000
1,2025-10-31 22:23:03.246999979,prc821916593,A20N,41.800644,-87.772897,1650.0,169.0,314.760270,1472.0,NaN,NaN,NaN,adsb,0.036291
2,2025-10-31 22:23:07.556999922,prc821916593,A20N,41.802933,-87.776392,1750.0,175.0,306.412064,1280.0,NaN,NaN,NaN,adsb,0.385658
3,2025-10-31 22:23:08.056999922,prc821916593,A20N,41.803165,-87.776865,1775.0,175.0,306.412064,1280.0,NaN,NaN,NaN,adsb,0.046949
4,2025-10-31 22:23:08.638999939,prc821916593,A20N,41.803397,-87.777376,1775.0,176.0,303.870811,1344.0,NaN,NaN,NaN,adsb,0.049637
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19876,2025-11-01 01:32:19.427000046,prc821916593,A20N,22.693909,-99.369640,36000.0,NaN,NaN,NaN,NaN,NaN,NaN,adsb,0.256391
19877,2025-11-01 01:32:20.496000051,prc821916593,A20N,22.691888,-99.370422,36000.0,NaN,NaN,NaN,NaN,NaN,NaN,adsb,0.238561
19878,2025-11-01 01:32:21.588000059,prc821916593,A20N,22.690018,-99.371188,36000.0,NaN,NaN,NaN,NaN,NaN,NaN,adsb,0.222343
19879,2025-11-01 01:32:23.341000080,prc821916593,A20N,22.686535,-99.372609,36000.0,NaN,NaN,NaN,NaN,NaN,NaN,adsb,0.413822


In [17]:
df_fuel.head(20)

,idx,flight_id,start,end,fuel_kg,seg_dataset,seg_flight_distance_km,seg_altitude_max,seg_altitude_min,seg_altitude_diff,seg_avg_rate_of_climb,seg_sd_rate_of_climb,seg_pos_count
0,0,prc770822360,2025-04-13 02:31:04.447,2025-04-13 03:01:04.487,2500.000000,train,66.385642,39025.0,38975.0,50.0,1.357576,35.783875,518.0
1,1,prc770822360,2025-04-13 03:01:04.487,2025-04-13 03:16:04.226,1200.000000,train,77.196913,39025.0,33900.0,5125.0,-1333.719457,1334.326818,597.0
2,2,prc770822360,2025-04-13 03:16:04.458,2025-04-13 03:46:04.417,2500.000000,train,58.862654,33900.0,24600.0,9300.0,-2326.970954,250.092447,484.0
3,3,prc770822360,2025-04-13 03:46:04.417,2025-04-13 04:01:04.540,1300.000000,train,28.196969,22200.0,17075.0,5125.0,-2208.000000,158.639052,284.0
4,4,prc770822360,2025-04-13 04:01:04.540,2025-04-13 04:16:04.263,600.000000,train,21.375931,17100.0,13000.0,4100.0,-2122.340611,80.086615,231.0
5,5,prc770831136,2025-04-12 21:30:33.782,2025-04-12 21:58:39.507,3049.952608,train,34.578844,10300.0,6875.0,3425.0,-872.489796,609.594005,485.0
6,6,prc770831136,2025-04-12 21:58:39.507,2025-04-12 22:02:40.552,355.162536,train,5.728733,6900.0,6200.0,700.0,-823.069767,102.099696,88.0
7,7,prc770831136,2025-04-12 22:02:40.552,2025-04-12 22:22:44.881,1930.941144,train,8.091503,3850.0,2800.0,1050.0,-947.582090,160.164028,136.0
8,8,prc770831136,2025-04-12 22:22:44.881,2025-04-12 22:38:54.572,1607.076456,train,5.821066,2800.0,1800.0,1000.0,-796.888889,84.902174,146.0
9,9,prc770831136,2025-04-12 22:38:54.572,2025-04-12 22:42:48.575,370.131072,train,3.121896,1825.0,1300.0,525.0,-695.138462,50.440521,87.0


In [18]:
df_merged = df_fuel.merge(df_flight, on='flight_id', how='left')


In [19]:
df_merged['seg_duration_hrs'] = (df_merged['end'] - df_merged['start']).dt.total_seconds() / 3600
df_merged['total_flight_duration_hrs'] = df_merged.groupby('flight_id')['seg_duration_hrs'].transform('sum')
df_merged['seg_duration_perc'] = df_merged['seg_duration_hrs'] / df_merged['total_flight_duration_hrs']

In [20]:
df_merged

,idx,flight_id,start,end,fuel_kg,seg_dataset,seg_flight_distance_km,seg_altitude_max,seg_altitude_min,seg_altitude_diff,...,great_circle_distance_km,total_flight_distance_km,altitude_max,altitude_min,altitude_diff,avg_rate_of_climb,sd_rate_of_climb,seg_duration_hrs,total_flight_duration_hrs,seg_duration_perc
0,0,prc770822360,2025-04-13 02:31:04.447,2025-04-13 03:01:04.487,2500.0,train,66.385642,39025.0,38975.0,50.0,...,10236.638941,10686.041654,36024.9216,349.9104,35675.0112,-157.532470,692.268730,0.500011,1.749884,0.285740
1,1,prc770822360,2025-04-13 03:01:04.487,2025-04-13 03:16:04.226,1200.0,train,77.196913,39025.0,33900.0,5125.0,...,10236.638941,10686.041654,36024.9216,349.9104,35675.0112,-157.532470,692.268730,0.249927,1.749884,0.142825
2,2,prc770822360,2025-04-13 03:16:04.458,2025-04-13 03:46:04.417,2500.0,train,58.862654,33900.0,24600.0,9300.0,...,10236.638941,10686.041654,36024.9216,349.9104,35675.0112,-157.532470,692.268730,0.499989,1.749884,0.285727
3,3,prc770822360,2025-04-13 03:46:04.417,2025-04-13 04:01:04.540,1300.0,train,28.196969,22200.0,17075.0,5125.0,...,10236.638941,10686.041654,36024.9216,349.9104,35675.0112,-157.532470,692.268730,0.250034,1.749884,0.142886
4,4,prc770822360,2025-04-13 04:01:04.540,2025-04-13 04:16:04.263,600.0,train,21.375931,17100.0,13000.0,4100.0,...,10236.638941,10686.041654,36024.9216,349.9104,35675.0112,-157.532470,692.268730,0.249923,1.749884,0.142823
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
193270,37451,prc821916593,2025-10-31 23:06:58.703,2025-10-31 23:17:08.526,NaN,final,132.458354,36050.0,35950.0,100.0,...,2741.031216,2425.341715,36050.0000,1650.0000,34400.0000,225.052924,619.452225,0.169395,1.776150,0.095372
193271,37452,prc821916593,2025-10-31 23:17:08.526,2025-10-31 23:22:13.468,NaN,final,65.860003,36025.0,36000.0,25.0,...,2741.031216,2425.341715,36050.0000,1650.0000,34400.0000,225.052924,619.452225,0.084706,1.776150,0.047691
193272,37453,prc821916593,2025-10-31 23:22:13.468,2025-10-31 23:27:18.458,NaN,final,65.860655,36025.0,35975.0,50.0,...,2741.031216,2425.341715,36050.0000,1650.0000,34400.0000,225.052924,619.452225,0.084719,1.776150,0.047698
193273,37454,prc821916593,2025-10-31 23:27:18.458,2025-10-31 23:52:43.483,NaN,final,331.650412,36050.0,35950.0,100.0,...,2741.031216,2425.341715,36050.0000,1650.0000,34400.0000,225.052924,619.452225,0.423618,1.776150,0.238504


In [21]:
list(df_merged.columns)

['idx',
 'flight_id',
 'start',
 'end',
 'fuel_kg',
 'seg_dataset',
 'seg_flight_distance_km',
 'seg_altitude_max',
 'seg_altitude_min',
 'seg_altitude_diff',
 'seg_avg_rate_of_climb',
 'seg_sd_rate_of_climb',
 'seg_pos_count',
 'flight_date',
 'aircraft_type',
 'takeoff',
 'landed',
 'origin_icao',
 'origin_name',
 'destination_icao',
 'destination_name',
 'dataset',
 'origin_longitude',
 'origin_latitude',
 'origin_elevation',
 'destination_longitude',
 'destination_latitude',
 'destination_elevation',
 'great_circle_distance_km',
 'total_flight_distance_km',
 'altitude_max',
 'altitude_min',
 'altitude_diff',
 'avg_rate_of_climb',
 'sd_rate_of_climb',
 'seg_duration_hrs',
 'total_flight_duration_hrs',
 'seg_duration_perc']

In [22]:
from pathlib import Path

out_path = Path(folder).expanduser() / 'df_merged_fin.pkl'
out_path.parent.mkdir(parents=True, exist_ok=True)

df_merged.to_pickle(out_path)
print(f"Saved df_merged to {out_path}")

Saved df_merged to /Users/joao/OneDrive - Embry-Riddle Aeronautical University/PRC Challenge 2025 - challenge data/final_phase/prc-2025-datasets/df_merged_fin.pkl


In [23]:
out_csv = Path(folder).expanduser() / 'df_merged_fin.csv'
out_csv.parent.mkdir(parents=True, exist_ok=True)
df_merged.to_csv(out_csv, index=False)
print(f"Saved df_merged to {out_csv}")

Saved df_merged to /Users/joao/OneDrive - Embry-Riddle Aeronautical University/PRC Challenge 2025 - challenge data/final_phase/prc-2025-datasets/df_merged_fin.csv
